<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning_(SSL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the SiameseNetwork class
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5),  # First convolutional layer
            nn.ReLU(),  # ReLU activation
            nn.MaxPool2d(2),  # Max pooling
            nn.Conv2d(64, 128, kernel_size=5),  # Second convolutional layer
            nn.ReLU(),  # ReLU activation
            nn.MaxPool2d(2),  # Max pooling
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),  # First fully connected layer
            nn.ReLU(),  # ReLU activation
            nn.Linear(256, 256),  # Second fully connected layer
        )

    def forward(self, x1, x2):
        output1 = self.cnn(x1)  # Process first image through CNN
        output1 = output1.view(output1.size(0), -1)  # Flatten the tensor
        output1 = self.fc(output1)  # Process through fully connected layers

        output2 = self.cnn(x2)  # Process second image through CNN
        output2 = output2.view(output2.size(0), -1)  # Flatten the tensor
        output2 = self.fc(output2)  # Process through fully connected layers

        return output1, output2  # Return the embeddings for both images

# Example usage
model = SiameseNetwork()
img1, img2 = torch.randn(32, 1, 28, 28), torch.randn(32, 1, 28, 28)  # Example input pairs (batch_size=32, channels=1, height=28, width=28)
output1, output2 = model(img1, img2)

# Print the shape of the outputs
print(output1.shape, output2.shape)  # Expected shapes: [32, 256], [32, 256]